In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/hmeq_clean.csv',index_col=0)
y = data['BAD']

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model_rf = RandomForestClassifier(n_estimators=200)
model_rf.fit(data.drop(columns=['BAD']), y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [4]:
data_pos = data.loc[data['BAD']==0]
data_pos.drop(columns=['BAD'],inplace=True)
data_pos = data_pos.T

/Users/luciasuelves/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
from scipy.spatial.distance import pdist, squareform

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data_pos.T, 'euclidean'))), 
                         index=data_pos.columns, columns=data_pos.columns)

distances.head()

,4,13,19,26,29,30,34,35,38,48,...,5950,5951,5952,5953,5954,5955,5956,5957,5958,5959
4,1.000000,0.000024,0.000099,0.000189,0.000009,0.000023,0.000159,0.000091,0.000084,0.000018,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
13,0.000024,1.000000,0.000020,0.000022,0.000014,0.000089,0.000021,0.000019,0.000019,0.000058,...,0.000012,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011
19,0.000099,0.000020,1.000000,0.000184,0.000008,0.000019,0.000116,0.000475,0.000410,0.000015,...,0.000010,0.000010,0.000010,0.000010,0.000009,0.000010,0.000010,0.000010,0.000009,0.000009
26,0.000189,0.000022,0.000184,1.000000,0.000009,0.000021,0.000138,0.000145,0.000131,0.000016,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
29,0.000009,0.000014,0.000008,0.000009,1.000000,0.000014,0.000008,0.000008,0.000008,0.000017,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009


In [7]:
tom = pd.Series({'LOAN': 50000,
 'MORTDUE': 47000,
 'VALUE': 20000,
 'YOJ': 5,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 200,
 'CLNO': 10,
 'DEBTINC': 0,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})

data_pos['TOM'] = tom

In [8]:
model_rf.predict(pd.DataFrame(tom).T)[0]

1

In [9]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data_pos.T, 'euclidean'))), 
                         index=data_pos.columns, columns=data_pos.columns)


similarities = distances['TOM'].sort_values(ascending=False)[1:]

## Este es el vector de similiaridad de TOM con el resto de usuarios respecto a sus
## visualizaciones comunes

similars = pd.DataFrame(similarities)

In [10]:

data_pos[["TOM",4523]]

,TOM,4523
LOAN,50000,23600.000000
MORTDUE,47000,69846.000000
VALUE,20000,189234.000000
YOJ,5,11.000000
DEROG,0,0.000000
DELINQ,0,0.000000
CLAGE,200,357.805062
NINQ,0,0.000000
CLNO,10,17.000000
DEBTINC,0,30.564280


In [11]:
cols = ['JOB_Self','JOB_Mgr','JOB_Other','JOB_Office','JOB_ProfExe',
        'REASON_DebtCon','REASON_HomeImp','DEROG','DELINQ']
bigger = ['MORTDUE','CLNO','DEBTINC','NINQ']
smaller = ['LOAN','CLAGE']

df_empty_2 = pd.DataFrame({title:[] for title in data_pos.index})
df_empty_3 = pd.DataFrame({title:[] for title in data_pos.index})
df_empty_4 = pd.DataFrame({title:[] for title in data_pos.index})
for i in similars.index:
    if all([data_pos[i][col] == tom[col] for col in cols]):
        if [data_pos[i][col] > tom[col] for col in bigger] == [True]*len(bigger):
            if data_pos[i]['YOJ'] < tom['YOJ']:
                if data_pos[i]['LOAN'] < tom['LOAN']:
                    print('Ask for a smaller loan')
                    df_empty_3 = pd.concat([df_empty_3,pd.DataFrame(data_pos[i]).T])
                else:
                    print('You can get your loan')
                    df_empty_4 = pd.concat([df_empty_4,pd.DataFrame(data_pos[i]).T])
            else:
                print('Work longer')
                df_empty_2 = pd.concat([df_empty_2,pd.DataFrame(data_pos[i]).T])

display(df_empty_2.sort_values(by='LOAN',ascending=False))
display(df_empty_3.sort_values(by='LOAN',ascending=False))
display(df_empty_4.sort_values(by='LOAN',ascending=False))

Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer


,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self
5929,71200.0,196601.0,289430.0,12.0,0.0,0.0,226.914307,2.0,25.0,29.220351,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5928,70800.0,192044.0,292380.0,10.0,0.0,0.0,221.608486,2.0,25.0,30.378548,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5927,70500.0,196690.0,288512.0,9.0,0.0,0.0,227.027136,2.0,24.0,29.582301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5926,70300.0,197761.0,294169.0,9.0,0.0,0.0,212.239725,2.0,24.0,28.907073,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5924,69700.0,192163.0,293901.0,10.0,0.0,0.0,222.147526,2.0,24.0,29.038236,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5925,69700.0,191972.0,294367.0,10.0,0.0,0.0,230.705613,2.0,25.0,28.201871,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5923,69600.0,198702.0,293252.0,10.0,0.0,0.0,208.299007,2.0,24.0,27.761644,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5922,69000.0,197587.0,295551.0,10.0,0.0,0.0,211.776391,3.0,24.0,30.356857,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5921,68700.0,197356.0,291242.0,10.0,0.0,0.0,217.801647,2.0,24.0,29.423058,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5920,68100.0,193298.0,288525.0,10.0,0.0,0.0,228.287295,2.0,25.0,30.395869,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self
4698,24800.0,98139.0,121696.0,3.0,0.0,0.0,208.523267,1.0,27.0,32.644265,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4658,24400.0,95843.0,123377.0,2.0,0.0,0.0,201.976756,1.0,26.0,32.903468,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4623,24200.0,90626.0,116601.0,2.0,0.0,0.0,226.116387,1.0,27.0,34.499938,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4576,23900.0,90244.0,120751.0,3.0,0.0,0.0,194.987252,1.0,26.0,35.406060,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4486,23400.0,89226.0,119604.0,3.0,0.0,0.0,221.681383,1.0,26.0,34.556201,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4484,23400.0,97865.0,120172.0,2.0,0.0,0.0,200.575752,1.0,26.0,36.398729,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4399,22900.0,95321.0,123082.0,3.0,0.0,0.0,205.441519,2.0,26.0,34.053491,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4261,22200.0,93947.0,116176.0,2.0,0.0,0.0,205.029409,2.0,26.0,34.939403,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4179,21800.0,90065.0,122335.0,3.0,0.0,0.0,212.695468,1.0,26.0,34.846919,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4188,21800.0,96709.0,123613.0,3.0,0.0,0.0,218.730333,2.0,27.0,35.874495,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self


In [12]:
tom = pd.Series({'LOAN': 50000,
 'MORTDUE': 47000,
 'VALUE': 200000,
 'YOJ': 2.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 200,
 'CLNO': 10,
 'DEBTINC': 5,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})
model_rf.predict(pd.DataFrame(tom).T)[0]

0

In [13]:
while model_rf.predict(pd.DataFrame(tom).T)[0] == 1:
    tom['LOAN'] *= 0.95 
    if tom['MORTDUE'] > 1000:
        tom['MORTDUE'] -= 1000
    else:
        tom['MORTDUE'] -= 1000
    tom['CLAGE'] += 1
    tom['YOJ'] += 0.1
    print(tom)
    print(model_rf.predict(pd.DataFrame(tom).T)[0])

In [14]:
data.loc[(data['CLAGE']!=0) & (data['CLNO']==0)]

,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self


In [15]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression()
model_lr.fit(data.drop(columns=['LOAN']),data['LOAN'])

/Users/luciasuelves/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/luciasuelves/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
tom = pd.Series({'BAD': 0,
 'MORTDUE': 47000,
 'VALUE': 200000,
 'YOJ': 2.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 200,
 'CLNO': 10,
 'DEBTINC': 5,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})

In [17]:
tom = pd.Series({'BAD': 1,
 'MORTDUE': 680000,
 'VALUE': 50000,
 'YOJ': 2.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 50,
 'CLNO': 10,
 'DEBTINC': 10,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})

In [18]:
model_lr.predict(pd.DataFrame(tom).T)[0]

34100

In [19]:
data.drop(columns=['LOAN']).head().shape

(5, 18)

In [20]:
model_lr2 = LogisticRegression()
model_lr2.fit(data.drop(columns=['BAD']),data['BAD'])

/Users/luciasuelves/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [21]:
tom = pd.Series({'LOAN': 196830.0,
 'MORTDUE': 680000,
 'VALUE': 50000,
 'YOJ': 2.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 50,
 'CLNO': 10,
 'DEBTINC': 10,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})

In [22]:
model_lr2.predict(pd.DataFrame(tom).T)[0]

0

In [23]:
#no me da el maximo

In [24]:
while model_lr2.predict(pd.DataFrame(tom).T)[0] == 1:
    tom['LOAN'] *= .9

In [25]:
tom['LOAN']

196830.0

In [26]:
#EN realidad es que el LOGREG tiene como linea de corte, el rf importa mas la combinacion de resultados

In [ ]:
#MAKE A MODEL TO PREDICT LOAN

In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
regr = RandomForestRegressor(max_depth=24, random_state=0)
X = data.drop(columns=['LOAN'])
y = data['LOAN']
regr.fit(X, y)

/Users/luciasuelves/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=24,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [29]:
regr_r = RandomForestRegressor(max_depth=24, random_state=0)
X = data.drop(columns=['BAD'])
y = data['BAD']
regr_r.fit(X, y)

/Users/luciasuelves/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=24,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [30]:
regr.score(X,y)

-2851468805.562098

In [31]:
tom = pd.Series({'BAD': 0,
 'MORTDUE': 680000,
 'VALUE': 50000,
 'YOJ': 2.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 50,
 'CLNO': 10,
 'DEBTINC': 10,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})

In [32]:
regr.predict(pd.DataFrame(tom).T)[0]

38090.0

In [33]:
tom = pd.Series({'LOAN': 38090,
 'MORTDUE': 680000,
 'VALUE': 50000,
 'YOJ': 2.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 50,
 'CLNO': 10,
 'DEBTINC': 10,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})

In [34]:
regr_r.predict(pd.DataFrame(tom).T)[0]

0.6

In [35]:
len(-model_lr2.coef_)

1

In [36]:
-model_lr2.intercept_

array([-0.00035834])

In [38]:
type(coefs)

pandas.core.series.Series

In [39]:
coefs.tolist()

[-9.67774391258109e-06,
 -2.5286119045019646e-06,
 4.114532494100516e-06,
 -0.010802104593539022,
 0.5159167434114761,
 0.703474390001064,
 -0.006386731131787327,
 0.1658941155856236,
 0.0035095879688550446,
 -0.05835354845492697,
 0.19662413826941103,
 0.5038795268755268,
 -0.025984290329547292,
 -0.49055752098596683,
 -0.010119765377436505,
 -0.26350971086595776,
 1.1078647707137237,
 0.3828094205717306]

In [40]:
tom = pd.Series({'LOAN': 196830.0,
 'MORTDUE': 680000,
 'VALUE': 50000,
 'YOJ': 2.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 50,
 'CLNO': 10,
 'DEBTINC': 10,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})
for i in range(12):
    tom[tom.keys()[i]] = coefs[i]

In [41]:
tom

LOAN             -0.000010
MORTDUE          -0.000003
VALUE             0.000004
YOJ              -0.010802
DEROG             0.515917
DELINQ            0.703474
CLAGE            -0.006387
CLNO              0.165894
DEBTINC           0.003510
NINQ             -0.058354
REASON_DebtCon    0.196624
REASON_HomeImp    0.503880
JOB_Mgr           0.000000
JOB_Office        0.000000
JOB_Other         0.000000
JOB_ProfExe       0.000000
JOB_Sales         0.000000
JOB_Self          1.000000
dtype: float64

In [43]:
crazy = LogisticRegression()
X = data.drop(columns=['BAD'])
y = data['BAD']
crazy.fit(X,y)

/Users/luciasuelves/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [62]:
coefs=-crazy.coef_[0]
type(coefs)

numpy.ndarray

In [45]:
tom = pd.Series({'LOAN': 196830.0,
 'MORTDUE': 680000,
 'VALUE': 50000,
 'YOJ': 2.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 50,
 'CLNO': 10,
 'DEBTINC': 10,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})
for i in range(12):
    tom[tom.keys()[i]] = coefs[i]

In [46]:
crazy.predict(pd.DataFrame(tom).T)[0]

0

In [47]:
intercept=-crazy.intercept_[0]
intercept

-0.000362854856852603

In [267]:
tom = pd.Series({   
 'LOAN': 50000.0,
 'MORTDUE': 60000,
 'VALUE': 50000,
 'YOJ': 2.0,
 'DEROG': 10,
 'DELINQ': 0,
 'CLAGE': 9,
 'CLNO': 10,
 'DEBTINC': 10,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})
crazy.predict(pd.DataFrame(tom).T)[0]

1

In [245]:
n=-crazy.coef_[0]
print(n)
#n = np.add(n,tom.values)
print(n)
n /= np.linalg.norm(n)

[-1.16389684e-06  6.00509352e-07 -3.14847136e-06 -1.41720683e-04
 -4.43687547e-03 -8.12587620e-03  4.03002786e-03 -6.06532344e-03
 -1.45619464e-02  6.33383988e-02  8.28347095e-05 -4.45689566e-04
 -2.82555668e-04  4.37216511e-04 -4.77758542e-04  3.02845253e-04
 -1.70771408e-04 -1.71831004e-04]
[-1.16389684e-06  6.00509352e-07 -3.14847136e-06 -1.41720683e-04
 -4.43687547e-03 -8.12587620e-03  4.03002786e-03 -6.06532344e-03
 -1.45619464e-02  6.33383988e-02  8.28347095e-05 -4.45689566e-04
 -2.82555668e-04  4.37216511e-04 -4.77758542e-04  3.02845253e-04
 -1.70771408e-04 -1.71831004e-04]


In [246]:
x = -intercept/n[0]
x*n[0]+intercept

0.0

In [247]:
x


-20.593592974708375

In [248]:
p0 = [0]*17
p0.insert(0,x)
p0

[-20.593592974708375, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [249]:
np.dot(p0,n)+intercept

0.0

In [250]:
p_ = tom.values - p0
dist_to_plane = np.dot(p_, n)

In [251]:
dist_to_plane

3.679571482860948

In [252]:
p_normal = np.dot(p_, n) * n
p_tangent = p_ - p_normal

In [253]:
closest_point = p_tangent + p0
closest_point

array([ 5.00000001e+04,  6.00000000e+04,  5.00000002e+04,  2.00789436e+00,
        1.02471501e+01,  4.52640879e-01,  9.77551278e+00,  1.03378606e+01,
        1.08111534e+01,  6.47182067e+00,  9.95385805e-01,  2.48265309e-02,
        1.57393790e-02, -2.43545509e-02,  2.66128896e-02, -1.68695828e-02,
        9.51258897e-03,  1.00957161e+00])

In [254]:
tom

LOAN              50000.0
MORTDUE           60000.0
VALUE             50000.0
YOJ                   2.0
DEROG                10.0
DELINQ                0.0
CLAGE                10.0
CLNO                 10.0
DEBTINC              10.0
NINQ                 10.0
REASON_DebtCon        1.0
REASON_HomeImp        0.0
JOB_Mgr               0.0
JOB_Office            0.0
JOB_Other             0.0
JOB_ProfExe           0.0
JOB_Sales             0.0
JOB_Self              1.0
dtype: float64

In [255]:
for i in range(12):
    tom[tom.keys()[i]] = closest_point[i]

In [256]:
crazy.predict(pd.DataFrame(tom).T)[0]

1

In [257]:
closest_point

array([ 5.00000001e+04,  6.00000000e+04,  5.00000002e+04,  2.00789436e+00,
        1.02471501e+01,  4.52640879e-01,  9.77551278e+00,  1.03378606e+01,
        1.08111534e+01,  6.47182067e+00,  9.95385805e-01,  2.48265309e-02,
        1.57393790e-02, -2.43545509e-02,  2.66128896e-02, -1.68695828e-02,
        9.51258897e-03,  1.00957161e+00])

In [258]:
#DEBERIA DAR 0
np.dot(tom.values,coefs) + intercept

-0.0003026979902107711

In [196]:
#NO ES UN PUNTO FACTIBLEEEE

In [259]:
#EL problema es que al dar tantos pasos se pierde precision y no se pasa

In [261]:
for i in range(12):
    tom[tom.keys()[i]] += 1

In [262]:
crazy.predict(pd.DataFrame(tom).T)[0]

0

In [263]:
tom

LOAN              50001.000065
MORTDUE           60000.999967
VALUE             50001.000175
YOJ                   3.007894
DEROG                11.247150
DELINQ                1.452641
CLAGE                10.775513
CLNO                 11.337861
DEBTINC              11.811153
NINQ                  7.471821
REASON_DebtCon        1.995386
REASON_HomeImp        1.024827
JOB_Mgr               0.000000
JOB_Office            0.000000
JOB_Other             0.000000
JOB_ProfExe           0.000000
JOB_Sales             0.000000
JOB_Self              1.000000
dtype: float64

In [320]:
tom = pd.Series({   
 'LOAN': 600000.0,
 'MORTDUE': 60000,
 'VALUE': 15000,
 'YOJ': 10.0,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 200,
 'CLNO': 10,
 'DEBTINC': 10,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':1,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':0})
crazy.predict(pd.DataFrame(tom).T)[0]

1

In [321]:
while crazy.predict(pd.DataFrame(tom).T)[0]==1:
    if tom['LOAN'] <= 100:
        break
    tom['LOAN'] -= 1000
    if tom['MORTDUE'] > 2000:
        tom['MORTDUE'] -= 2000
    else: 
        tom['MORTDUE'] = 0
    if tom['MORTDUE'] > 2000:
        tom['VALUE'] += 2000
    else: 
        tom['VALUE'] += tom['MORTDUE']
    tom['YOJ'] += 0.1
    tom['CLAGE'] += 1

In [322]:
tom

LOAN              536000.0
MORTDUE                0.0
VALUE              73000.0
YOJ                   16.4
DEROG                  0.0
DELINQ                 0.0
CLAGE                264.0
CLNO                  10.0
DEBTINC               10.0
NINQ                   0.0
REASON_DebtCon         1.0
REASON_HomeImp         0.0
JOB_Mgr                1.0
JOB_Office             0.0
JOB_Other              0.0
JOB_ProfExe            0.0
JOB_Sales              0.0
JOB_Self               0.0
dtype: float64

In [ ]:
#Tengo tres sistemas de consejos: recomendacion, punto más cercano, while loop
#No ha funcionado lo de predecir el loan y luego pasar el test. 
# HOY: Organizar, revisar codigo. Incluir recomendaciones en flask.
# Mañana: Unsupervised learning, clustering.